<a href="https://colab.research.google.com/github/DarioRugg/KickLearning/blob/main/historic_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from os.path import join
path = join('drive', 'MyDrive', 'Project', 'Data', 'complete')

In [65]:
import pandas as pd
import numpy as np
df = pd.concat([pd.read_csv(join(path, f'file_00{"0" +str(i) if i in range(10) else i}.csv')) for i in range(17)], ignore_index=True)

In [67]:
df.drop(columns=['time'], inplace=True)

In [68]:
historical_vars =  ['historical_backers', 'historical_launch', 'historical_state', 'historical_pledges']

In [69]:
df.columns = df.columns[:-4].tolist() + historical_vars

In [71]:
def get_statistics(entry):
  
  if type(entry['historical_launch']) == str:
    delta_launch = entry['launched_at'] - np.array(eval(entry['historical_launch']))
    entry['historical_delta_launch'] = delta_launch
    success_rate = np.mean(list(map(lambda x: x == 'successful', eval(entry['historical_state']))))
    entry['historical_success_rate'] = success_rate

    for var_name in ['historical_backers', 'historical_delta_launch', 'historical_pledges']:
      var = entry[var_name]
      if type(var) == str:
        var_array = np.array(eval(var))
      else:
        var_array = np.array(var)
      entry[f"{var_name}_mean"] = var_array.mean()
      entry[f"{var_name}_std"] = var_array.std()
      entry[f"{var_name}_median"] = np.median(var_array)
      entry[f"{var_name}_max"] = var_array.max()
      entry[f"{var_name}_min"] = var_array.min()
    entry[f"historical_projects"] = len(var_array)
  else:
    entry[f"historical_projects"] = 0
  return entry
hist_df = df.apply(get_statistics, axis=1).drop(columns=historical_vars+['historical_delta_launch'])

In [74]:
hist_df.to_csv(join(path, 'complete_historical_variables.csv'), index=False)

In [77]:
hist_df.shape

(250593, 33)

In [76]:
hist_df.columns

Index(['backers_count', 'category', 'country', 'created_at', 'creator_id',
       'currency', 'deadline', 'disable_communication', 'fx_rate', 'goal',
       'historical_backers_max', 'historical_backers_mean',
       'historical_backers_median', 'historical_backers_min',
       'historical_backers_std', 'historical_delta_launch_max',
       'historical_delta_launch_mean', 'historical_delta_launch_median',
       'historical_delta_launch_min', 'historical_delta_launch_std',
       'historical_pledges_max', 'historical_pledges_mean',
       'historical_pledges_median', 'historical_pledges_min',
       'historical_pledges_std', 'historical_projects',
       'historical_success_rate', 'id', 'launched_at', 'project_url',
       'state_changed_at', 'sub_category', 'year'],
      dtype='object')